In [1]:
from urllib.request import urlopen
from PIL import Image
import timm
import torch

img = Image.open(urlopen(
    'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
))

model = timm.create_model('mobilenetv3_small_100.lamb_in1k', pretrained=True)
model = model.eval()

data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

output = model(transforms(img).unsqueeze(0))

top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=5)

c:\Users\Justin Clyde\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(top5_class_indices, top5_probabilities)

tensor([[968, 969, 927, 960, 505]]) tensor([[19.5942, 11.7789,  8.0567,  6.7019,  4.9821]],
       grad_fn=<TopkBackward0>)


In [4]:
import torch
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

test_dir = './dataset/test'
train_dir = './dataset/train'
val_dir = './dataset/val'

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_ds = ImageFolder(train_dir, transform=transform)
val_ds = ImageFolder(val_dir, transform=transform)
test_ds = ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

Preprocess the images on each subfolder


In [ ]:
from PIL import Image, ImageFilter
import os

test_dir = './dataset/test'
train_dir = './dataset/train'
val_dir = './dataset/val'

dir_lst = [test_dir, train_dir, val_dir]

filters = [ImageFilter.BoxBlur(5), ImageFilter.SHARPEN, ImageFilter.SMOOTH]
filtername = ["BLUR", "SHARP", "SMOOTH"]

for dir in dir_lst:
    for d in os.listdir(dir):
        for j in os.listdir(f'{dir}/{d}'):
            # print(f'{dir}/{d}/{j}')
            # break
            with Image.open(f'{dir}/{d}/{j}') as im:
                for val in range(3):
                    im = im.convert("RGB")
                    new_image = im.filter(filters[val])
                    new_image.save(f'{dir}/{d}/{j[:-4]}_{filtername[val]}')
            break
        # break
    # break


./dataset/test/OP01-001/OP01-001.png


Next step is to create a train and test method

In [4]:
from datasets import load_dataset

imagenet = load_dataset("timm/imagenet-1k-wds", split="train")  # downloads the full dataset
print(imagenet[0])

DatasetNotFoundError: Dataset 'timm/imagenet-1k-wds' is a gated dataset on the Hub. Visit the dataset page at https://huggingface.co/datasets/timm/imagenet-1k-wds to ask for access.

In [ ]:
from transformers import TrainingArguments, Trainer

trainingargs = TrainingArguments(
    output_dir='./model',
    max_steps=100,
    eval_strategy="epoch",
    per_device_train_batch_size=12,
)

trainer = Trainer(
    model=model,
    args=trainingargs,
    train_dataset=dataset["train"],
    eval_dataset=dataset["eval"]
)

trainer.train()